In [130]:
# %%
import sys

sys.path.append("../../")

In [131]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

from multi_tp.utils import LANGUAGES, get_model_name_path, get_suffix, performance_file_v2_tmpl
from multi_tp.models_ids import *

system_role = "normal"
translator_provider_forward = "google"
translator_provider_backward = "google"
analysis_backend_model_version = "meta-llama/Meta-Llama-3.1-8B-Instruct"
add_paraphrase = False
country = None

lang="en"


criteria = [
    "Species_Humans_Bin_Choice",
    "Age_Young_Bin_Choice",
    "Fitness_Fit_Bin_Choice",
    "Gender_Female_Bin_Choice",
    "SocialValue_High_Bin_Choice",
    "Utilitarianism_More_Bin_Choice",
]
custom_order = ["Species_Humans_Bin_Choice", 
                "Utilitarianism_More_Bin_Choice",
                "Age_Young_Bin_Choice",
                "SocialValue_High_Bin_Choice",
                "Fitness_Fit_Bin_Choice",
                "Gender_Female_Bin_Choice"
                ]


model_version = LLAMA_3_1_70B

all_data = {}
for model_version in MODELS:
    in_path = performance_file_v2_tmpl.format(
                model_version=get_model_name_path(model_version),
                system_role=system_role,
                lang=lang,
                suffix=get_suffix(add_paraphrase, country),
                translator_provider_forward=translator_provider_forward,
                translator_provider_backward=translator_provider_backward,
                analysis_backend_model_version=get_model_name_path(
                    analysis_backend_model_version
                ),
            )
    in_path= os.path.join("../../", in_path)
    # check if the file exists
    if not os.path.exists(in_path):
        print(f"File {in_path} does not exist")
        continue
    # Assuming you have the CSV file in the same directory
    data = pd.read_csv(in_path)

    data.index = data["criterion"]

    index = [d for d in data.index if d in criteria]
    data = data.loc[index]
    data = data[["criterion", "percentage"]]

    data["percentage"] = data["percentage"] / 100

    data.reset_index(drop=True, inplace=True)
    data['criterion']= pd.Categorical(data['criterion'], categories=custom_order, ordered=True)
    data = data.sort_values('criterion')
    all_data[model_version] = data


File ../../data/performance/B=meta-llama_Meta-Llama-3.1-8B-Instruct/microsoft_Phi-3.5-MoE-instruct_normal_en+google_tr+google_performance_v2.csv does not exist


In [132]:
humans = pd.read_csv("../../data/human/human_preferences_by_lang_converted.csv")
humans_to_our = {"Age": "Age_Young_Bin_Choice", "Fitness": "Fitness_Fit_Bin_Choice", "Gender": "Gender_Female_Bin_Choice" , "Social Status": "SocialValue_High_Bin_Choice", "Species": "Species_Humans_Bin_Choice", "No. Characters": "Utilitarianism_More_Bin_Choice"}
humans["Label"] = humans["Label"].apply(lambda x: humans_to_our[x])
humans = humans.rename(columns={"Label": "criterion"})
humans = humans[["criterion", lang]]
humans.rename(columns={lang: "percentage"}, inplace=True)
humans["percentage"] = humans["percentage"] / 100
humans['criterion']= pd.Categorical(humans['criterion'], categories=custom_order, ordered=True)
humans = humans.sort_values('criterion')
humans

,criterion,percentage
5,Species_Humans_Bin_Choice,0.797445
3,Utilitarianism_More_Bin_Choice,0.753526
0,Age_Young_Bin_Choice,0.724156
4,SocialValue_High_Bin_Choice,0.661781
1,Fitness_Fit_Bin_Choice,0.576128
2,Gender_Female_Bin_Choice,0.562544


In [133]:
for model_version in MODELS:
    if model_version not in all_data:
        continue
    # Create the figure
    fig = make_subplots(rows=1, cols=1, shared_yaxes=True)


    fig.add_trace(go.Bar(
        y=all_data[model_version]['criterion'],
        x=all_data[model_version]['percentage'],
        orientation='h',
        name=get_pretty_name(model_version),
        marker_color='#0077ad',
        marker_line_color='#0077ad',
        marker_line_width=2,
        opacity=0.7,
        width=0.3
    ))



    fig.add_trace(go.Scatter(
        y=humans['criterion'],
        x=humans['percentage'],
        mode='markers',
        name="Human Preference",
        marker=dict(size=20, color='white', line=dict(color='#ad0077', width=2)),  # Increased size
        showlegend=True
    ))

    # Customize the layout
    fig.update_layout(
        title='',
        xaxis=dict(
            title='Pr',
            tickvals=[0, 0.25, 0.5, 0.75, 1],
            ticktext=['100% Favoring the Left', '75-25', '50-50', '25-75', '100% Favoring the Right'],
            range=[-0.1, 1.1]
        ),
        yaxis=dict(
            categoryorder='array',
            categoryarray=custom_order[::-1],  # Reverse the order for bottom-to-top
            tickvals=list(range(len(custom_order))),
            ticktext=[""] * len(custom_order),  # Empty labels as before
            tickmode='array',
            range=[-0.5, 5.5], autorange=False
        ),
        height=600,
        width=1350,
        plot_bgcolor='white'
    )

    # Add right-side labels
    right_labels = [
        'Sparing humans',
        'Sparing more characters',
        'Sparing the young',
        'Sparing higher status',
        'Sparing the fit',
        'Sparing females'
    ][::-1]

    right_icons=[
        "humans",
        "more_character",
        "young",
        "higher_status",
        "fit",
        "female"
    ][::-1]

    for i, label in enumerate(right_labels):
        fig.add_annotation(
            x=1.06, y=i,
            xref='paper', yref='y',
            text=label,
            showarrow=False,
            xanchor='left',
            align='left',
            font=dict(size=14)
        )

        fig.add_layout_image(
            dict(
                source=f"./icons/icons_tradeoff/{right_icons[i]}.png",
                xref="paper",
                yref="y",
                x=0.95,
                y=i,
                sizex=0.5,
                sizey=0.5,
                xanchor="left",
                yanchor="middle"
            )
        )

    # Add left-side labels
    left_labels = [
        'Sparing pets',
        'Sparing fewer characters',
        'Sparing the elderly',
        'Sparing lower status',
        "Sparing the large",
        'Sparing males'
    ][::-1]

    left_icons=[
        "pet",
        "fewer",
        "elderly",
        "lower_status",
        "large",
        "male"
    ][::-1]

    for i, label in enumerate(left_labels):

        fig.add_annotation(
            x=-0.02, y=i,
            xref='paper', yref='y',
            text=label,
            showarrow=False,
            xanchor='right',
            align='right',
            font=dict(size=14)
        )

        fig.add_layout_image(
            dict(
                source=f"./icons/icons_tradeoff/{left_icons[i]}.png",
                xref="paper",
                yref="y",
                x=-0.01,
                y=i,
                sizex=0.5,
                sizey=0.5,
                xanchor="left",
                yanchor="middle"
            )
        )

    # Update axes
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', title="Preference")
    fig.update_yaxes(showgrid=False)

    fig.update_layout(
        height=800,
        width=1200,
        margin=dict(l=250, r=250, t=50, b=50),
        )

    fig.update_layout(
            legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="center", x=0.5),
            showlegend=True,
        )

    fig.update_layout(
        title="",
        font_family="Times New Roman",
        font_size=12,
        title_font_size=12,
        margin=dict(l=160, r=170, t=50, b=50),
        height=400,
        width=800,
    )

    os.makedirs("./imgs/preference_normal_model/", exist_ok=True)
    fig.write_image(f"./imgs/preference_normal_model/{get_pretty_name(model_version)}.png")
    fig.write_image(f"./imgs/preference_normal_model/{get_pretty_name(model_version)}.pdf")
    # Show the plot
    fig.show()

In [134]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Assuming all_data, humans, custom_order, get_pretty_name, LLAMA_3_1_70B, and LLAMA_3_1_8B are defined

# Combine data into a single DataFrame
df = pd.DataFrame({
    'criterion': all_data[LLAMA_3_1_70B]['criterion'],
    'LLAMA_3_1_70B': all_data[LLAMA_3_1_70B]['percentage'],
    'LLAMA_3_1_8B': all_data[LLAMA_3_1_8B]['percentage']
})

# Create the figure
fig = make_subplots(rows=1, cols=1, shared_yaxes=True)

# Add grouped bar trace with smaller width
fig.add_trace(go.Bar(
    y=df['criterion'],
    x=df['LLAMA_3_1_70B'],
    orientation='h',
    name=get_pretty_name(LLAMA_3_1_70B),
    marker_color='#0077ad',
    opacity=0.7,
    width=0.15  # Reduced width
))

fig.add_trace(go.Bar(
    y=df['criterion'],
    x=df['LLAMA_3_1_8B'],
    orientation='h',
    name=get_pretty_name(LLAMA_3_1_8B),
    marker_color='#00ad77',
    opacity=0.7,
    width=0.15  # Reduced width
))

# Add markers for human preferences with increased width
fig.add_trace(go.Scatter(
    y=humans['criterion'],
    x=humans['percentage'],
    mode='markers',
    name="Human Preference",
    marker=dict(size=20, color='white', line=dict(color='#ad0077', width=2)),  # Increased size
    showlegend=True
))

# Customize the layout
fig.update_layout(
    barmode='group',
    bargap=0.7,  # Increase space between bar groups
    title='',
    xaxis=dict(
        title='Preference',
        tickvals=[0, 0.25, 0.5, 0.75, 1],
        ticktext=['100% Favoring the Left', '75-25', '50-50', '25-75', '100% Favoring the Right'],
        range=[-0.1, 1.1]
    ),
    yaxis=dict(
        categoryorder='array',
        categoryarray=custom_order[::-1],  # Reverse the order for bottom-to-top
        tickvals=list(range(len(custom_order))),
        ticktext=[""] * len(custom_order),  # Empty labels as before
        tickmode='array',
        range=[-0.5, 5.5], autorange=False
    ),
    height=800,
    width=1200,
    plot_bgcolor='white'
)

# Add right-side labels
right_labels = [
    'Sparing humans',
    'Sparing more characters',
    'Sparing the young',
    'Sparing higher status',
    'Sparing the fit',
    'Sparing females'
][::-1]

right_icons = [
    "humans",
    "more_character",
    "young",
    "higher_status",
    "fit",
    "female"
][::-1]

for i, label in enumerate(right_labels):
    fig.add_annotation(
        x=1.06, y=i,
        xref='paper', yref='y',
        text=label,
        showarrow=False,
        xanchor='left',
        align='left',
        font=dict(size=14)
    )

    fig.add_layout_image(
        dict(
            source=f"./icons/icons_tradeoff/{right_icons[i]}.png",
            xref="paper",
            yref="y",
            x=0.95,
            y=i,
            sizex=0.5,
            sizey=0.5,
            xanchor="left",
            yanchor="middle"
        )
    )

# Add left-side labels
left_labels = [
    'Sparing pets',
    'Sparing fewer characters',
    'Sparing the elderly',
    'Sparing lower status',
    "Sparing the large",
    'Sparing males'
][::-1]

left_icons = [
    "pet",
    "fewer",
    "elderly",
    "lower_status",
    "large",
    "male"
][::-1]

for i, label in enumerate(left_labels):
    fig.add_annotation(
        x=-0.02, y=i,
        xref='paper', yref='y',
        text=label,
        showarrow=False,
        xanchor='right',
        align='right',
        font=dict(size=14)
    )

    fig.add_layout_image(
        dict(
            source=f"./icons/icons_tradeoff/{left_icons[i]}.png",
            xref="paper",
            yref="y",
            x=-0.01,
            y=i,
            sizex=0.5,
            sizey=0.5,
            xanchor="left",
            yanchor="middle"
        )
    )

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', title="Preference")
fig.update_yaxes(showgrid=False)

fig.update_layout(
    margin=dict(l=250, r=250, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="center", x=0.5),
    showlegend=True,
)

fig.update_layout(
    title="",
    font_family="Times New Roman",
    font_size=12,
    title_font_size=12,
    margin=dict(l=160, r=170, t=50, b=50),
    height=400,
    width=800,
)

os.makedirs("./imgs/preference_normal/", exist_ok=True)
fig.write_image("./imgs/preference_normal/llama_3_1.png")
fig.write_image("./imgs/preference_normal/llama_3_1.pdf")
# Show the plot
fig.show()